In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [10]:
# Завантаження даних
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [11]:
# Нормалізація даних
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

**Базова Модель**

---


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [13]:
from tensorflow.keras import optimizers
# Компіляція моделі
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
    )

In [14]:
# # Тренування моделі
# history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    callbacks=[reduce_lr]
)

Epoch 1/20
750/750 [==============================] - 10s 10ms/step - loss: 0.4626 - accuracy: 0.8406 - val_loss: 0.3088 - val_accuracy: 0.8869 - lr: 0.0010
Epoch 2/20
750/750 [==============================] - 7s 9ms/step - loss: 0.2904 - accuracy: 0.8953 - val_loss: 0.2770 - val_accuracy: 0.8932 - lr: 0.0010
Epoch 3/20
750/750 [==============================] - 7s 10ms/step - loss: 0.2502 - accuracy: 0.9107 - val_loss: 0.2202 - val_accuracy: 0.9212 - lr: 0.0010
Epoch 4/20
750/750 [==============================] - 7s 9ms/step - loss: 0.2241 - accuracy: 0.9193 - val_loss: 0.2096 - val_accuracy: 0.9251 - lr: 0.0010
Epoch 5/20
750/750 [==============================] - 7s 10ms/step - loss: 0.2044 - accuracy: 0.9250 - val_loss: 0.2172 - val_accuracy: 0.9208 - lr: 0.0010
Epoch 6/20
750/750 [==============================] - 7s 9ms/step - loss: 0.1867 - accuracy: 0.9315 - val_loss: 0.2207 - val_accuracy: 0.9203 - lr: 0.0010
Epoch 7/20
750/750 [==============================] - 7s 10ms/step

In [15]:
# Оцінка моделі
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")

313/313 [==============================] - 1s 4ms/step - loss: 0.2186 - accuracy: 0.9388
Test Accuracy: 93.88%
